Imports

In [12]:
!pip install boto3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [13]:
import os
import json
import re
from datetime import datetime
from bs4 import BeautifulSoup
from io import BytesIO
import boto3
from pyspark.sql import SparkSession

# Connection to MinIO

In [3]:
spark = SparkSession.builder \
    .appName("PySpark Access Logs Minio") \
    .master("spark://172.29.16.102:7077") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://172.29.16.105:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "bdenggroup3") \
    .config("spark.hadoop.fs.s3a.secret.key", "bdenggroup3") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 19:27:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/16 19:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/16 19:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/16 19:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/16 19:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/16 19:27:28 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


# Parser

## HTML Extraction Helper
fool

In [4]:
def extract_fool_article_text(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')
    article_body = soup.find("div", class_="article-body")
    if article_body:
        tags = article_body.find_all(['p', 'h2'])
        flat_text = ' '.join(tag.get_text(strip=True) for tag in tags)
        return re.sub(r'\s+', ' ', flat_text).strip()
    return ""


benzinga

In [5]:
def extract_benzinga_article_text(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')
    article_body = soup.find("div", id="article-body")
    if article_body:
        tags = article_body.find_all(['p', 'h2', 'li'])
        flat_text = ' '.join(tag.get_text(strip=True) for tag in tags)
        return re.sub(r'\s+', ' ', flat_text).strip()
    return ""

zacks

In [ ]:
def extract_zacks_article_text(html: str) -> str:
    soup = BeautifulSoup(html, 'html.parser')
    article_body = soup.find("div", id="comtext")

    if article_body:
        tags = article_body.find_all(['p', 'h2', 'li'])
        flat_text = ' '.join(tag.get_text(strip=True) for tag in tags)
        flat_text = re.sub(r'-{5,}.*?-{5,}', '', flat_text)  # Remove ad block content
        return re.sub(r'\s+', ' ', flat_text).strip()

    return ""


dynamic dispatcher


In [6]:
def extract_article_text_by_source(url: str, html: str) -> str:
    if "fool.com" in url:
        return extract_fool_article_text(html)
    elif "benzinga.com" in url:
        return extract_benzinga_article_text(html)
    elif "zacks.com" in url:
        return extract_zacks_article_text(html)
    else:
        return ""


full file processing (read, parse, write)

In [9]:
def process_json_file(s3_client, bucket: str, key: str) -> str:
    try:
        obj = s3_client.get_object(Bucket=bucket, Key=key)
        data = json.load(obj['Body'])

        html = data.get("html", "")
        url = data.get("url", "")
        scraping_timestamp = data.get("timestamp", "")

        article_text = extract_article_text_by_source(url, html)
        parsed_data = {
            "url": url,
            "scrapingTimestamp": scraping_timestamp,
            "parsingTimestamp": datetime.utcnow().isoformat() + "Z",
            "articleText": article_text
        }

        new_key = key.replace("scrape_raw_", "scrape_cleaned_")
        s3_client.put_object(
            Bucket=bucket,
            Key=new_key,
            Body=json.dumps(parsed_data, ensure_ascii=False, indent=2).encode("utf-8"),
            ContentType="application/json"
        )

        return f"✅ {key} → {new_key}"

    except Exception as e:
        return f"❌ Error processing {key}: {e}"


Initializing S3 Client & Batch processing

In [10]:
s3 = boto3.client(
    's3',
    endpoint_url='http://172.29.16.105:9000',
    aws_access_key_id='bdenggroup3',
    aws_secret_access_key='bdenggroup3'
)

bucket_name = "bdenggroup3"
response = s3.list_objects_v2(Bucket=bucket_name)

for obj in response.get("Contents", []):
    key = obj["Key"]
    if key.startswith("scrape_raw_") and key.endswith(".json"):
        result = process_json_file(s3, bucket_name, key)
        print(result)

✅ scrape_raw_0.json → scrape_cleaned_0.json
✅ scrape_raw_1.json → scrape_cleaned_1.json
✅ scrape_raw_10.json → scrape_cleaned_10.json
✅ scrape_raw_100.json → scrape_cleaned_100.json
✅ scrape_raw_1000.json → scrape_cleaned_1000.json
✅ scrape_raw_10000.json → scrape_cleaned_10000.json
✅ scrape_raw_10001.json → scrape_cleaned_10001.json
✅ scrape_raw_10002.json → scrape_cleaned_10002.json
✅ scrape_raw_10003.json → scrape_cleaned_10003.json
✅ scrape_raw_10004.json → scrape_cleaned_10004.json
✅ scrape_raw_10005.json → scrape_cleaned_10005.json
✅ scrape_raw_10006.json → scrape_cleaned_10006.json
✅ scrape_raw_10007.json → scrape_cleaned_10007.json
✅ scrape_raw_10008.json → scrape_cleaned_10008.json
✅ scrape_raw_10009.json → scrape_cleaned_10009.json
✅ scrape_raw_1001.json → scrape_cleaned_1001.json
✅ scrape_raw_10010.json → scrape_cleaned_10010.json
✅ scrape_raw_10011.json → scrape_cleaned_10011.json
✅ scrape_raw_10012.json → scrape_cleaned_10012.json
✅ scrape_raw_10013.json → scrape_cleaned_1

In [11]:
spark.stop()